## Bedrock model integration with Langchain Agents

Certain applications demand an adaptable sequence of calls to language models and various utilities depending on user input. The Agent interface enables such flexibility for these applications. An agent has availability to a range of resources and selects which ones to utilize based on the user input. Agents are capable of using multiple tools and utilizing the output of one tool as the input for the next.  

There are two primary categories of agents:

- Action agents: At each interval, determine the subsequent action utilizing the outputs of all previous actions. 
- Plan-and-execute agents: Determine the complete order of actions initially, then implement them all without updating the plan.

In this notebook, we will demonstrate the use `plan-and-execute` agents along with `Zero-shot ReAct` which i an action agent and uses the [`ReAct`](https://arxiv.org/pdf/2205.00445.pdf) framework to select the appropriate tool based exclusively on the tool's description. It requires you provide the description of each tool. 

## Architecture
![](./images/arch-agents.png)

## Setup

In [ ]:
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

In [ ]:
%pip install google-search-results

In [ ]:
import json
import os
import sys
import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

In [ ]:
model_parameter = {"temperature": 0.0, "top_p": .5, "max_tokens_to_sample": 2000}

## Using ReAct: Synergizing Reasoning and Acting in Language Models Framework
Large language models can generate both explanations for their reasoning and task-specific responses in an alternating fashion. 

Producing reasoning explanations enables the model to infer, monitor, and revise action plans, and even handle unexpected scenarios. The action step allows the model to interface with and obtain information from external sources such as knowledge bases or environments.

The ReAct framework could enable large language models to interact with external tools to obtain additional information that results in more accurate and fact-based responses.

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.llms.bedrock import Bedrock
from langchain import LLMMathChain
from langchain.utilities import SerpAPIWrapper

In [ ]:
from langchain.llms.bedrock import Bedrock
from typing import Optional, List, Any
from langchain.callbacks.manager import CallbackManagerForLLMRun

class BedrockModelWrapper(Bedrock):
    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        prompt = "\n\nHuman: " + prompt + "\n\nAssistant:"   ## Satisfy Bedrock-Claude prompt requirements
        return super()._call(prompt, stop, run_manager, **kwargs)

In [ ]:
llm = BedrockModelWrapper(model_id="anthropic.claude-instant-v1", client=boto3_bedrock, model_kwargs=model_parameter)

tools = load_tools(["serpapi", "llm-math"], llm=llm)
react_agent = initialize_agent(tools, 
                               llm, 
                               agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
                               verbose=True,
                            #    max_iteration=2,
                            #    return_intermediate_steps=True,
                            #    handle_parsing_errors=True,
                               )


In [ ]:
prompt_template = """Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do, Also try to follow steps mentioned above
Action: the action to take, should be one of ["serpapi", "llm-math"]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Question: {input}

Assistant:
{agent_scratchpad}"""

In [ ]:
react_agent.agent.llm_chain.prompt.template=prompt_template

In [ ]:
question = "What is Amazon SageMaker? What is the launch year multiplied by 2"

In [ ]:
react_agent("\n\nHuman:" + question + "\n\nAssistant:")

## Generating the tool code
Could you use GenAI to author the code in the tools? Sure! With caution, there is some variation in the code and 
may not be safe for execution without a human review.

The following example uses the code generated by Claude to generate the tool. It replaces the EC2Search tool above with the boto python code.

In [ ]:
import xml.etree.ElementTree as ET
from IPython.display import display, Markdown, Latex

prompt_data = """
Human: You are an AI python code generator. You write really great code.

Write a python function named list_tagged_instances with one parameter named tagname. 

The function queries the boto3 library to return a list all of the EC2 instances that have a tag equal to the tagname parameter. 

return the code inside <code></code>.
Assistant:"""

body = json.dumps({"prompt": prompt_data, "max_tokens_to_sample": 500})
modelId = "anthropic.claude-instant-v1"  
accept = "application/json"
contentType = "application/json"

response = boto3_bedrock.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())

tree = ET.ElementTree(ET.fromstring(response_body.get("completion")))
python_code = tree.getroot().text

display(Markdown(f'```{python_code}```'))

exec(python_code)



## Executing the code
If the code above looks reasonable, we can use it to run our agent.

**Note - for this to work, you need an EC2 instance in this account with a tag named 'delete' and any value in the tag. It is case senstitive, so prior to executing this, create an EC2 instance and set it to the stopped state.**

In [ ]:
for tool in tools:
    if tool.name == "EC2Search":
        tool.func = list_tagged_instances
        
llm = BedrockModelWrapper(model_id="anthropic.claude-instant-v1", client=boto3_bedrock, model_kwargs=model_parameter)

react_agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

question = """Human: Please list my EC2 instances with the a tag delete. 
Next, patch each of the instances and record the patch change record in the CMDB with a change type of 'PATCH'. 
Finally, for each of the instances stop the instance and tell me how many were stopped. 
Assistant:"""

result = react_agent.run(question)

print(f"{result}")

## Another variation - Search for an instance that does not exist
The prompt changes to search for EC2 instances with a tag that does not exist, so no instances should be returned.

In [ ]:
question = """Human: Please list my EC2 instances with the a tag doesnotexist. 
Next, patch each of the instances and record the patch change record in the CMDB with a change type of 'PATCH'. 
Finally, for each of the instances stop the instance and tell me how many were stopped. 
Assistant:"""

result = react_agent(question)

print(f"{result}")

## Database Tools
A common use of an agent is to look up a record in a database. It would not be practical to include the full database in the context, so you can provide tools that perform actions against the datebase that eliminates hallucinations while maintining the conversational interactions.

### SQL Database Agent
Langachain has a SQL Database agent for demonstrating how to ask questions of a DB to get answers. For details, read this document: https://python.langchain.com/docs/integrations/toolkits/sql_database

The agent will load the schema of the DB into context and generate SQL statements based on natural language questions. The SQL statement is then executed against the database and the results returned.

### Data Agents
While the SQL Database agent is useful for data exploration and generating queries, there are also cases where you want to 
For specific entities, a tool can be created to pull data from the database to provide context next steps in the prompt.

The following example will simulate a DB query for a customer in the customer table. Replace this code with a lookup ib DynamoDB or a relation database.

In [ ]:
customer_table=[
  {
    "id": 1, 
    "first_name": "John", 
    "last_name": "Doe",
    "age": 35,
    "postal_code": "90210"
  },
  {  
    "id": 2,
    "first_name": "Jane",
    "last_name": "Smith", 
    "age": 27,
    "postal_code": "12345"
  },
  {
    "id": 3, 
    "first_name": "Bob",
    "last_name": "Jones",
    "age": 42,
    "postal_code": "55555"
  },
  {
    "id": 4,
    "first_name": "Sara", 
    "last_name": "Miller",
    "age": 29, 
    "postal_code": "13579"
  },
  {
    "id": 5,
    "first_name": "Mark",
    "last_name": "Davis",
    "age": 31,
    "postal_code": "02468"
  },
  {
    "id": 6,
    "first_name": "Laura",
    "last_name": "Wilson",
    "age": 24,
    "postal_code": "98765" 
  },
  {
    "id": 7,
    "first_name": "Steve",
    "last_name": "Moore",
    "age": 36,
    "postal_code": "11223"
  },
  {
    "id": 8,
    "first_name": "Michelle",
    "last_name": "Chen",
    "age": 22,
    "orders": [
        {
            "order_id": 1,
            "description": "An order of 1 dozen pencils"
        },
        {
            "order_id": 2,
            "description": "An order of 2 markers"
        }
    ],
    "postal_code": "33215"
  },
  {
    "id": 9,
    "first_name": "David",
    "last_name": "Lee",
    "age": 29,
    "postal_code": "99567"
  },
  {
    "id": 10,
    "first_name": "Jessica",
    "last_name": "Brown",
    "age": 18, 
    "postal_code": "43210"
  }
]

def customer_lookup(id):
    print(f"search by customer {id}")
    for customer in customer_table:
        if customer["id"] == int(id):
            print(f"found customer {id} {customer}")
            return customer
        
    return None

tools.append(Tool.from_function(
        name="CustomerLookup",
        func=customer_lookup,  # Mock Function, replace with an api call
        description="Use this when you need to lookup a customer by id."
    ))

In [ ]:
react_agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

question = """\n\nHuman: write one sentence summary about the information you know about the customer with an id of 2.

\n\nAssistant: Here is the one sentence summary: """

result = react_agent.run(question)

print(f"{result}")